In [1]:
import os

import requests
import json
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
index_name = "conversations"
index_path = "../indexes/conversations"
embeddings = OpenAIEmbeddings()
# db = FAISS.from_texts(texts=['9'], embedding=embeddings)
# db.save_local(folder_path=index_path, index_name=index_name) 
# db = FAISS.load_local(folder_path=index_path, index_name=index_name, embeddings=embeddings)


In [3]:
def gpt_system_user(
    system_message: str, user_message: str, model: str = "gpt-3.5-turbo"
):
    """Para usar en notebooks"""
    # Hyper 2
    # POST https://yoizenia.openai.azure.com/openai/deployments/GPT35Turbo/chat/completions

    try:
        response = requests.post(
            url="https://api.openai.com/v1/chat/completions",
            params={
                "temperature": "0",
            },
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {OPENAI_API_KEY}",
            },
            data=json.dumps(
                {
                    "model": model,
                    "messages": [
                        {"content": system_message, "role": "system"},
                        {"content": user_message, "role": "user"},
                    ]
                }
            ),
            timeout=10,
        )
        choices = response.json()["choices"]
        answer = {
            "status_code": response.status_code,
            "text": choices[0]["message"],
        }
        return answer

    except requests.exceptions.RequestException:
        print("HTTP Request failed")
        return None


In [4]:
condensed_contexts =  [
        {
          "messages": [
              {
                "role": "user",
                "content": "Voy a sacar un pasaje para la familia de buenos aires a cordoba"
              },
              {
                "role": "system",
                "content": "quienes son los pasajeros?"
              },
              {
                "role": "user",
                "content": "Ana es mi hija mayor, tiene 20 años. Juan es mi hijo menor, tiene 10 años. Mi esposa se llama Maria y tiene 40 años. Yo me llamo Pedro y tengo 45 años"
              },
              {
                "role": "system",
                "content": "en que fecha quiere viajar?"
              },
              {
                "role": "user",
                "content": "del 20 de agosto al 30 de agosto"
              },
              {
                  "role": "system",
                  "content": "perfecto, salen el 20 de agosto a las 10:00 y vuelven el 30 de agosto a las 18:00. El precio total es de 1000 dolares. Desea confirmar la compra?"
              },
              {
                  "role": "user",
                  "content": "si"
              },
              {
                  "role": "system",
                  "content": "Gracias por su compra"
              }
           ],
          "metadata": {
            "session_id": "1",
            "person_id": "20",
            "context_id": "1",
            "created_at": "20200825T12:00:00.000Z",
          },
        },
        {
            "messages": [
                {
                "role": "user",
                "content": "Quiero subir de categoria en el programa de viajero frecuente"
                },
                {
                    "role": "system",
                    "content": "que categoria tiene actualmente?"
                },
                { 
                  "role": "user",
                  "content": "soy categoria 1"
                },
                {
                  "role": "system",
                  "content": "para subir de categoria debe tener 1000 millas. Cuantas millas tiene actualmente?"
                },
                {
                  "role": "user",
                  "content": "tengo 500 millas"
                },
                {
                  "role": "system",
                  "content": "puede comprar 500 millas por 100 dolares. Desea comprarlas?"
                },
                {
                  "role": "user",
                  "content": "si"
                },
                {
                  "role": "system",
                  "content": "Gracias por su compra. Ahora tiene 1000 millas y es categoria 2"
                }
            ],
            "metadata": {
                "session_id": "1",
                "person_id": "20",
                "context_id": "2",
                "created_at": "20220825T18:00:00.000Z",                
              }
        }
  ]


In [5]:
db = FAISS.from_texts(texts=['notebook-11'], embedding=embeddings)
for context in condensed_contexts:
        # Join all messages in one string by a comma
        messages = ''.join([str(message) for message in context['messages']])
        # print(messages)
        # print(context['metadata'])
        extraction = gpt_system_user(
            system_message="""
              Extract every meaningful information about the user from the provided conversation.
              Ask yourself, what happened in the conversation?
              What data about the user did you learn?
              At what date and time was this data gathered?

              Create a summary of everything you learned about the user.
              """,
            user_message=f" METADATA: ```{context['metadata']} CONVERSATION: ```{messages}``` ```",
            model="gpt-3.5-turbo",
        )
        if extraction['status_code'] != 200:
            print(f"Error: {extraction['text']}")
            continue
        else:
            print(f"{extraction['text']['content']} \n\n\n///////////////////////////\n\n\n")
            db.add_texts(
                texts=[extraction['text']['content']], 
                metadatas=[context['metadata']],
            )

Summary:
This conversation took place on August 25th, at 12:00 PM UTC. The user, Pedro, is planning to book a ticket for his family from Buenos Aires to Cordoba. He mentions that his daughter, Ana, is 20 years old, his son Juan is 10 years old, and his wife Maria is 40 years old. Pedro himself is 45 years old. He wants to travel from August 20th to August 30th. The system confirms the details and provides the departure and return times, as well as the total price of $1000. Pedro confirms the purchase, and the conversation ends with a system message thanking him for the purchase. 


///////////////////////////



Data gathered:

- Date and time: August 25, 2022, at 18:00:00 UTC
- User's session ID: 1
- User's person ID: 20
- User's context ID: 2

Summary:

From the conversation, we learned the following information about the user:

- The user wants to upgrade their category in a frequent traveler program.
- The user's current category is category 1.
- The user has 500 miles.
- The user 

Veamos como se lleva con recuentos.

In [6]:
user_input_3 = "cuantos viajes hice con ustedes?"
filter = {"person_id": "20"}
db_results_3 = db.similarity_search_with_score(query=user_input_3, embeddings=embeddings, filter=filter, k=1)
context_3 = "{ message: " + db_results_3[0][0].page_content + ", metadata: " + str(db_results_3[0][0].metadata) + " }"
print(f"CONTEXT:{context_3}")
system_prompt_3 = f"""
You are a personal assistant. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.
```{context_3}```

"""
answer_3 = gpt_system_user(system_message=system_prompt_3, user_message=user_input_3)
answer_3["text"]["content"]


CONTEXT:{ message: Summary:
This conversation took place on August 25th, at 12:00 PM UTC. The user, Pedro, is planning to book a ticket for his family from Buenos Aires to Cordoba. He mentions that his daughter, Ana, is 20 years old, his son Juan is 10 years old, and his wife Maria is 40 years old. Pedro himself is 45 years old. He wants to travel from August 20th to August 30th. The system confirms the details and provides the departure and return times, as well as the total price of $1000. Pedro confirms the purchase, and the conversation ends with a system message thanking him for the purchase., metadata: {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'} }


'Ha realizado un viaje con nosotros. Recientemente reservó un boleto para viajar de Buenos Aires a Córdoba del 20 al 30 de agosto.'

Bien, igual hubo uno solo y hay que ver el tema cantidad de chunks.

In [8]:
user_input_3 = "cuantos hijos mios conoces?"
filter = {"person_id": "20"}
db_results_3 = db.similarity_search_with_score(query=user_input_3, embeddings=embeddings, filter=filter, k=1)
context_3 = "{ message: " + db_results_3[0][0].page_content + ", metadata: " + str(db_results_3[0][0].metadata) + " }"
print(f"CONTEXT:{context_3}")
system_prompt_3 = f"""
You are a personal assistant. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.
```{context_3}```

"""
answer_3 = gpt_system_user(system_message=system_prompt_3, user_message=user_input_3, model="gpt-4")
answer_3["text"]["content"]


CONTEXT:{ message: Summary:
This conversation took place on August 25th, at 12:00 PM UTC. The user, Pedro, is planning to book a ticket for his family from Buenos Aires to Cordoba. He mentions that his daughter, Ana, is 20 years old, his son Juan is 10 years old, and his wife Maria is 40 years old. Pedro himself is 45 years old. He wants to travel from August 20th to August 30th. The system confirms the details and provides the departure and return times, as well as the total price of $1000. Pedro confirms the purchase, and the conversation ends with a system message thanking him for the purchase., metadata: {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'} }


'Conozco a dos de tus hijos, Ana y Juan. Ana tiene 20 años y Juan tiene 10 años, según la información de nuestra conversación el 25 de agosto.'

Trae bien los hijos pero sigue mal con las edades. y es GPT 4. 

In [9]:
user_input_3 = "cuantos hijos mios conoces y que edades tienen?"
filter = {"person_id": "20"}
db_results_3 = db.similarity_search_with_score(query=user_input_3, embeddings=embeddings, filter=filter, k=1)
context_3 = "{ message: " + db_results_3[0][0].page_content + ", metadata: " + str(db_results_3[0][0].metadata) + " }"
print(f"CONTEXT:{context_3}")
system_prompt_3 = f"""
You are a personal assistant. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.
```{context_3}```

"""
answer_3 = gpt_system_user(system_message=system_prompt_3, user_message=user_input_3, model="gpt-4")
answer_3["text"]["content"]


CONTEXT:{ message: Summary:
This conversation took place on August 25th, at 12:00 PM UTC. The user, Pedro, is planning to book a ticket for his family from Buenos Aires to Cordoba. He mentions that his daughter, Ana, is 20 years old, his son Juan is 10 years old, and his wife Maria is 40 years old. Pedro himself is 45 years old. He wants to travel from August 20th to August 30th. The system confirms the details and provides the departure and return times, as well as the total price of $1000. Pedro confirms the purchase, and the conversation ends with a system message thanking him for the purchase., metadata: {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'} }


'Conozco a dos de tus hijos. De nuestra conversación del 25 de agosto de 2020, sé que Ana tenía 20 años y Juan tenía 10 años. Por lo tanto, hoy, 13 de septiembre de 2023, Ana tendría 23 años y Juan tendría 13 años.'

Buen razonamiento de gpt4

In [10]:
user_input_3 = "cuantos hijos mios conoces y que edades tienen?"
filter = {"person_id": "20"}
db_results_3 = db.similarity_search_with_score(query=user_input_3, embeddings=embeddings, filter=filter, k=1)
context_3 = "{ message: " + db_results_3[0][0].page_content + ", metadata: " + str(db_results_3[0][0].metadata) + " }"
print(f"CONTEXT:{context_3}")
system_prompt_3 = f"""
You are a personal assistant. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.
```{context_3}```

"""
answer_3 = gpt_system_user(system_message=system_prompt_3, user_message=user_input_3)
answer_3["text"]["content"]


CONTEXT:{ message: Summary:
This conversation took place on August 25th, at 12:00 PM UTC. The user, Pedro, is planning to book a ticket for his family from Buenos Aires to Cordoba. He mentions that his daughter, Ana, is 20 years old, his son Juan is 10 years old, and his wife Maria is 40 years old. Pedro himself is 45 years old. He wants to travel from August 20th to August 30th. The system confirms the details and provides the departure and return times, as well as the total price of $1000. Pedro confirms the purchase, and the conversation ends with a system message thanking him for the purchase., metadata: {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'} }


'Según nuestra conversación anterior, usted tiene dos hijos. Ana tiene 20 años y Juan tiene 10 años.'

GPT 3.5 sigue sin razonar. 

In [11]:
user_input_3 = "con quienes viaje a cordoba?"
filter = {"person_id": "20"}
db_results_3 = db.similarity_search_with_score(query=user_input_3, embeddings=embeddings, filter=filter, k=1)
context_3 = "{ message: " + db_results_3[0][0].page_content + ", metadata: " + str(db_results_3[0][0].metadata) + " }"
print(f"CONTEXT:{context_3}")
system_prompt_3 = f"""
You are a personal assistant. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.
```{context_3}```

"""
answer_3 = gpt_system_user(system_message=system_prompt_3, user_message=user_input_3, model="gpt-4")
answer_3["text"]["content"]


CONTEXT:{ message: Summary:
This conversation took place on August 25th, at 12:00 PM UTC. The user, Pedro, is planning to book a ticket for his family from Buenos Aires to Cordoba. He mentions that his daughter, Ana, is 20 years old, his son Juan is 10 years old, and his wife Maria is 40 years old. Pedro himself is 45 years old. He wants to travel from August 20th to August 30th. The system confirms the details and provides the departure and return times, as well as the total price of $1000. Pedro confirms the purchase, and the conversation ends with a system message thanking him for the purchase., metadata: {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'} }


'Viajaste a Córdoba con tu familia: tu hija Ana, quien tenía 20 años, tu hijo Juan, quien tenía 10 años, y tu esposa María, que tenía 40 años.'

In [12]:
user_input_3 = "cuanto habia costado el viaje a cordoba?"
filter = {"person_id": "20"}
db_results_3 = db.similarity_search_with_score(query=user_input_3, embeddings=embeddings, filter=filter, k=1)
context_3 = "{ message: " + db_results_3[0][0].page_content + ", metadata: " + str(db_results_3[0][0].metadata) + " }"
print(f"CONTEXT:{context_3}")
system_prompt_3 = f"""
You are a personal assistant. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.
```{context_3}```

"""
answer_3 = gpt_system_user(system_message=system_prompt_3, user_message=user_input_3, model="gpt-4")
answer_3["text"]["content"]


CONTEXT:{ message: Summary:
This conversation took place on August 25th, at 12:00 PM UTC. The user, Pedro, is planning to book a ticket for his family from Buenos Aires to Cordoba. He mentions that his daughter, Ana, is 20 years old, his son Juan is 10 years old, and his wife Maria is 40 years old. Pedro himself is 45 years old. He wants to travel from August 20th to August 30th. The system confirms the details and provides the departure and return times, as well as the total price of $1000. Pedro confirms the purchase, and the conversation ends with a system message thanking him for the purchase., metadata: {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'} }


'El viaje de Buenos Aires a Cordoba que Pedro reservó el 25 de agosto costó un total de $1000.'

Interesante. No reconoce que esta hablando con pedro y lo trata como 3ra persona.

In [13]:
user_input_3 = "cuanto habia costado el viaje a cordoba?"
filter = {"person_id": "20"}
db_results_3 = db.similarity_search_with_score(query=user_input_3, embeddings=embeddings, filter=filter, k=1)
context_3 = "{ message: " + db_results_3[0][0].page_content + ", metadata: " + str(db_results_3[0][0].metadata) + " }"
print(f"CONTEXT:{context_3}")
system_prompt_3 = f"""
You are the personal assistant of the user. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.
```{context_3}```

"""
answer_3 = gpt_system_user(system_message=system_prompt_3, user_message=user_input_3)
answer_3["text"]["content"]


CONTEXT:{ message: Summary:
This conversation took place on August 25th, at 12:00 PM UTC. The user, Pedro, is planning to book a ticket for his family from Buenos Aires to Cordoba. He mentions that his daughter, Ana, is 20 years old, his son Juan is 10 years old, and his wife Maria is 40 years old. Pedro himself is 45 years old. He wants to travel from August 20th to August 30th. The system confirms the details and provides the departure and return times, as well as the total price of $1000. Pedro confirms the purchase, and the conversation ends with a system message thanking him for the purchase., metadata: {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'} }


'El costo del viaje a Córdoba era de $1000.'

In [37]:
user_input = "cuantas millas tengo?"
filter = {"person_id": "20"}
db_results = db.similarity_search_with_score(query=user_input, embeddings=embeddings, filter=filter, k=2)
context_3 = "{ message: " + db_results_3[1][0].page_content + ", metadata: " + str(db_results_3[1][0].metadata) + " }"
context_k2 = [f'\n {db_results[i][0].page_content} {db_results[i][0].metadata}\n---------------\n\n' for i in range(len(db_results))]
context = context_k2
for c in context:
    print(c)
system_prompt_3 = f"""
You are the personal assistant of the user. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.

```{context}```

"""
answer_3 = gpt_system_user(system_message=system_prompt_3, user_message=user_input_3)
answer_3["text"]["content"]



 Summary:
This conversation took place on August 25th, at 12:00 PM UTC. The user, Pedro, is planning to book a ticket for his family from Buenos Aires to Cordoba. He mentions that his daughter, Ana, is 20 years old, his son Juan is 10 years old, and his wife Maria is 40 years old. Pedro himself is 45 years old. He wants to travel from August 20th to August 30th. The system confirms the details and provides the departure and return times, as well as the total price of $1000. Pedro confirms the purchase, and the conversation ends with a system message thanking him for the purchase. {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'}
---------------



 Data gathered:

- Date and time: August 25, 2022, at 18:00:00 UTC
- User's session ID: 1
- User's person ID: 20
- User's context ID: 2

Summary:

From the conversation, we learned the following information about the user:

- The user wants to upgrade their category in a frequent traveler progr

'Según la última información recopilada el 25 de agosto de 2022, usted tiene 1000 millas en su cuenta.'

In [45]:
user_input = "cuantas millas tenia yo antes de la ultima compra?"
filter = {"person_id": "20"}
db_results = db.similarity_search_with_score(query=user_input, embeddings=embeddings, filter=filter, k=2)
context_k2 = [f'\n {db_results[i][0].page_content} {db_results[i][0].metadata}\n---------------\n\n' for i in range(len(db_results))]
context = context_k2
for c in context:
    print(c)
system_prompt = f"""
You are the personal assistant of the user. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.

```{context}```

"""
answer = gpt_system_user(system_message=system_prompt, user_message=user_input, model="gpt-4")
answer["text"]["content"]



 Summary:
This conversation took place on August 25th, at 12:00 PM UTC. The user, Pedro, is planning to book a ticket for his family from Buenos Aires to Cordoba. He mentions that his daughter, Ana, is 20 years old, his son Juan is 10 years old, and his wife Maria is 40 years old. Pedro himself is 45 years old. He wants to travel from August 20th to August 30th. The system confirms the details and provides the departure and return times, as well as the total price of $1000. Pedro confirms the purchase, and the conversation ends with a system message thanking him for the purchase. {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'}
---------------



 Data gathered:

- Date and time: August 25, 2022, at 18:00:00 UTC
- User's session ID: 1
- User's person ID: 20
- User's context ID: 2

Summary:

From the conversation, we learned the following information about the user:

- The user wants to upgrade their category in a frequent traveler progr

'Antes de tu última compra, tenías 500 millas.'

Mezclando data y fechas de dos contextos

In [47]:
user_input = "cuantas millas tenia en el momento de hacer el viaje a cordoba?"
filter = {"person_id": "20"}
db_results = db.similarity_search_with_score(query=user_input, embeddings=embeddings, filter=filter, k=2)
context_k2 = [f'\n {db_results[i][0].page_content} {db_results[i][0].metadata}\n---------------\n\n' for i in range(len(db_results))]
context = context_k2
for c in context:
    print(c)
system_prompt = f"""
You are the personal assistant of the user. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.

```{context}```

"""
answer = gpt_system_user(system_message=system_prompt, user_message=user_input, model="gpt-4")
answer["text"]["content"]



 Summary:
This conversation took place on August 25th, at 12:00 PM UTC. The user, Pedro, is planning to book a ticket for his family from Buenos Aires to Cordoba. He mentions that his daughter, Ana, is 20 years old, his son Juan is 10 years old, and his wife Maria is 40 years old. Pedro himself is 45 years old. He wants to travel from August 20th to August 30th. The system confirms the details and provides the departure and return times, as well as the total price of $1000. Pedro confirms the purchase, and the conversation ends with a system message thanking him for the purchase. {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'}
---------------



 Data gathered:

- Date and time: August 25, 2022, at 18:00:00 UTC
- User's session ID: 1
- User's person ID: 20
- User's context ID: 2

Summary:

From the conversation, we learned the following information about the user:

- The user wants to upgrade their category in a frequent traveler progr

'El viaje a Cordoba se realizó el 20 de agosto de 2020, y la información sobre las millas del usuario se registró el 25 de agosto de 2022. Por lo tanto, en el momento del viaje a Cordoba, no se tiene registrada la cantidad de millas que el usuario tenía.'

In [44]:
user_input = "que categoria soy?"
filter = {"person_id": "20"}
db_results = db.similarity_search_with_score(query=user_input, embeddings=embeddings, filter=filter, k=2)
context_3 = "{ message: " + db_results_3[1][0].page_content + ", metadata: " + str(db_results_3[1][0].metadata) + " }"
context_k2 = [f'\n {db_results[i][0].page_content} {db_results[i][0].metadata}\n---------------\n\n' for i in range(len(db_results))]
context = context_k2
for c in context:
    print(c)
system_prompt = f"""
You are the personal assistant of the user. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.

```{context}```

"""
answer_3 = gpt_system_user(system_message=system_prompt, user_message=user_input, model="gpt-4")
answer_3["text"]["content"]



 Summary:
This conversation took place on August 25th, at 12:00 PM UTC. The user, Pedro, is planning to book a ticket for his family from Buenos Aires to Cordoba. He mentions that his daughter, Ana, is 20 years old, his son Juan is 10 years old, and his wife Maria is 40 years old. Pedro himself is 45 years old. He wants to travel from August 20th to August 30th. The system confirms the details and provides the departure and return times, as well as the total price of $1000. Pedro confirms the purchase, and the conversation ends with a system message thanking him for the purchase. {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'}
---------------



 Data gathered:

- Date and time: August 25, 2022, at 18:00:00 UTC
- User's session ID: 1
- User's person ID: 20
- User's context ID: 2

Summary:

From the conversation, we learned the following information about the user:

- The user wants to upgrade their category in a frequent traveler progr

'Según la conversación del 25 de agosto de 2022, usted es de la categoría 2 en el programa de viajero frecuente.'

In [48]:
user_input = "cuanto pagué para llegar a nivel 2?"
filter = {"person_id": "20"}
db_results = db.similarity_search_with_score(query=user_input, embeddings=embeddings, filter=filter, k=2)
context_3 = "{ message: " + db_results_3[1][0].page_content + ", metadata: " + str(db_results_3[1][0].metadata) + " }"
context_k2 = [f'\n {db_results[i][0].page_content} {db_results[i][0].metadata}\n---------------\n\n' for i in range(len(db_results))]
context = context_k2
for c in context:
    print(c)
system_prompt = f"""
You are the personal assistant of the user. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.

```{context}```

"""
answer_3 = gpt_system_user(system_message=system_prompt, user_message=user_input, model="gpt-4")
answer_3["text"]["content"]



 Summary:
This conversation took place on August 25th, at 12:00 PM UTC. The user, Pedro, is planning to book a ticket for his family from Buenos Aires to Cordoba. He mentions that his daughter, Ana, is 20 years old, his son Juan is 10 years old, and his wife Maria is 40 years old. Pedro himself is 45 years old. He wants to travel from August 20th to August 30th. The system confirms the details and provides the departure and return times, as well as the total price of $1000. Pedro confirms the purchase, and the conversation ends with a system message thanking him for the purchase. {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'}
---------------



 Data gathered:

- Date and time: August 25, 2022, at 18:00:00 UTC
- User's session ID: 1
- User's person ID: 20
- User's context ID: 2

Summary:

From the conversation, we learned the following information about the user:

- The user wants to upgrade their category in a frequent traveler progr

'Para alcanzar la categoría 2 en el programa de viajero frecuente, pagaste $100 para comprar 500 millas, según la conversación que tuvimos el 25 de agosto de 2022.'